# Setting Konfigurasi dan Data Loading

In [74]:
from bokeh.io import output_notebook, show, reset_output,output_file
import bokeh
from bokeh.plotting import figure
import numpy as np
import pandas as pd
from datetime import datetime
from openpyxl import load_workbook
from bokeh.models import ColumnDataSource, HoverTool, LinearAxis, Range1d, Label, LabelSet
from bokeh.transform import dodge
from bokeh.palettes import Spectral6
from bokeh.transform import factor_cmap
from bokeh.models.widgets import DataTable, TableColumn
from bokeh.layouts import row

tgl="26 April"
output_notebook()


Loading BokehJS ...

In [75]:
# JUMLAH KASUS COVID19 DI SUMATERA UTARA PER HARI 
df=pd.read_excel("../Covid19SumutApr.xlsx","Kab-Kota")
sumut=df.groupby(['Tanggal']).sum()
##################################### Ubah Tanggal 
path = r"D:\Proyek\2020\covidUSU\tabel\hsl26pr.xlsx"
writer = pd.ExcelWriter(path, engine = 'xlsxwriter')
sumut.to_excel(writer, sheet_name = 'sumut',index=True)
df['BPS'] = '0' + df['BPS'].astype(str)

In [92]:
source = ColumnDataSource(data=dict(tgl=df.Tanggal,kode=df.BPS, daerah=df.Daerah,odp=df.ODP,pdp=df.PDP,positif=df.Positif,sembuh=df.Sembuh,meninggal=df.Meninggal,))
columns = [
    TableColumn(field="tgl", title="Tanggal"),
    TableColumn(field="kode", title="Kode"),
    TableColumn(field="daerah", title="Kabupaten/Kota"),
    TableColumn(field="odp", title="ODP"),
    TableColumn(field="pdp", title="PDP"),
    TableColumn(field="positif", title="Positif"),
    TableColumn(field="sembuh", title="Sembuh"),
    TableColumn(field="meninggal", title="Meninggal"),
]
tabelsumut = DataTable(source=source, columns=columns, width=750, height=400, index_position=None)
show(tabelsumut)

# PENGOLAHAN DATA SE PROVINSI

In [77]:
# PERTAMBAHAN JUMLAH KASUS COVID19 DI SUMATERA UTARA PER HARI 
jumlah=sumut[["ODP","PDP","Positif","Sembuh","Meninggal"]].diff()
persen=sumut[["ODP","PDP","Positif","Sembuh","Meninggal"]].pct_change()
persen = persen.rename({"ODP": "rODP","PDP": "rPDP","Positif": "rPositif","Sembuh": "rSembuh","Meninggal": "rMeninggal",}, axis='columns')
recover=(sumut['Sembuh']/sumut['Positif'])*100
fatal=(sumut['Meninggal']/sumut['Positif'])*100
frr=(sumut['Sembuh']/sumut['Meninggal'])*100


### Penambahan Kasus dan Kecepatan Penambahannya

In [78]:
olah = pd.concat([jumlah,persen,recover,fatal,frr], axis=1, sort=False)
olah = olah.rename({0: "recoverPos",1: "fatalPos", 2: "rec_fat"}, axis='columns')
olah.to_excel(writer, sheet_name = 'olah', index=True)
olah=olah.reset_index()

sourceolah = ColumnDataSource(data=dict(tanggal=olah.Tanggal, odp=olah.ODP,pdp=olah.PDP,positif=olah.Positif,sembuh=olah.Sembuh,
            meninggal=olah.Meninggal,rodp=olah.rODP,rpdp=olah.rPDP,rpositif=olah.rPositif,rsembuh=olah.rSembuh,rmeninggal=olah.rMeninggal,
            recoverpos=olah.recoverPos,fatalpos=olah.fatalPos,rec_fatal=olah.rec_fat))
columnsolah = [
    TableColumn(field="odp", title="+an ODP"),
    TableColumn(field="pdp", title="+an PDP"),
    TableColumn(field="positif", title="+an Positif"),
    TableColumn(field="sembuh", title="+an Sembuh"),
    TableColumn(field="meninggal", title="+an Meninggal"),
    TableColumn(field="rodp", title="laju ODP"),
    TableColumn(field="rpdp", title="laju PDP"),
    TableColumn(field="rpositif", title="laju Positif"),
    TableColumn(field="rsembuh", title="laju Sembuh"),
    TableColumn(field="rmeninggal", title="laju Meninggal"),
    TableColumn(field="recoverpos", title="Recover Positif"),
    TableColumn(field="fatalpos", title="Fatal Positif"),
    TableColumn(field="rec_fatal", title="Recovery/Fatal"),
]
tabelsumutolah = DataTable(source=sourceolah, columns=columnsolah, width=1000, height=400, index_position=None)
show(tabelsumutolah)


In [79]:
# GRAFIK PERKEMBANGAN KASUS KASUS COVID19 DI SUMATERA UTARA PER HARI 
sumut.reset_index(level=0, inplace=True)
sumut['Tanggal']= pd.to_datetime(sumut['Tanggal']) 

In [80]:
line_plot = figure(plot_width=900, 
            plot_height=450, 
            title="Perkembangan Dampak Covid19 di Prov. Sumatera Utara per "+tgl +"2020",
            x_axis_type='datetime',x_axis_label="tanggal", y_axis_label="org")

line_plot.y_range = Range1d(start=0,end=max([sumut.PDP.max(),sumut.Sembuh.max(),sumut.Meninggal.max()]))
line_plot.extra_y_ranges = {"ODP": Range1d(start=0,end=sumut['ODP'].max())}
line_plot.add_layout(LinearAxis(y_range_name="ODP",axis_label="org - ODP"), 'right')
line_plot.xaxis.axis_label_text_font_size = '9pt'
line_plot.xaxis.axis_label_text_font_size = '9pt'


line_plot.line(sumut.Tanggal,sumut.ODP,legend_label="ODP",line_width=1,line_color='green',y_range_name="ODP")
line_plot.line(sumut.Tanggal,sumut.PDP,legend_label="PDP",line_width=1,line_color='darkorange')
line_plot.line(sumut.Tanggal,sumut.Positif,legend_label="Positif",line_width=1,line_color='red')
line_plot.line(sumut.Tanggal,sumut.Sembuh,legend_label="Sembuh",line_width=1,line_color='royalblue')
line_plot.line(sumut.Tanggal,sumut.Meninggal,legend_label="Meninggal",line_width=1,line_color='black')
line_plot.legend.label_text_font_size = '9pt'

line_plot.add_tools(HoverTool(
        tooltips=[('Tanggal', "@x{%m-%d-%Y}"),('Jumlah', "@y{0}")],
        formatters={
            'x': 'datetime',
            'Value': 'printf',
        }))
show(line_plot)

In [81]:
tambah=olah[['Tanggal','PDP','Positif']]
tambah=tambah.dropna()
olahsource=ColumnDataSource(data=tambah)
b = figure(x_range=tambah.Tanggal,y_range=(tambah[['PDP','Positif']].values.min() -5, tambah[['PDP','Positif']].values.max() + 5), 
           plot_height=400, title="Penambahan Kasus",width=900)
labels15=LabelSet(x=dodge('Tanggal', -0.25, range=b.x_range),y='PDP',text='PDP',source=olahsource,text_align='center',text_font_size="6Pt")
labels16=LabelSet(x=dodge('Tanggal', 0.25, range=b.x_range),y='Positif',text='Positif',source=olahsource,text_align='center',text_font_size="6Pt")

vbar1=b.vbar(x=dodge('Tanggal',-0.2,range=b.x_range),top="PDP",width=0.4, source=olahsource,legend_label='PDP',
       color="yellow")
b.vbar(x=dodge('Tanggal', 0.2, range=b.x_range), top='Positif', width=0.4, source=olahsource,legend_label='Positif',
       color="red")

b.add_layout(labels15)
b.add_layout(labels16)
b.xaxis.major_label_orientation = "vertical"
show(b)

In [85]:
rasio=olah[['Tanggal','recoverPos','fatalPos','rec_fat']]
rasio.reset_index(level=0, inplace=True)
rasio['Tanggal']= pd.to_datetime(rasio['Tanggal']) 
rasiosource=ColumnDataSource(data=rasio)

C:\Users\asus\anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [86]:
line_rasio = figure(plot_width=900, 
            plot_height=450, 
            title="Perkembangan Tingkat Kesembuhan dan Fatalitas Dampak Covid19 di Prov. Sumatera Utara per "+tgl +"2020",
            x_axis_type='datetime',x_axis_label="tanggal", y_axis_label="%")

#line_rasio.y_range = Range1d(start=0,end=max([rasio.rec_fat.max()]))
line_rasio.xaxis.axis_label_text_font_size = '9pt'
line_rasio.xaxis.axis_label_text_font_size = '9pt'

line_rasio.multi_line(xs = [rasio.Tanggal,rasio.Tanggal] , ys = [rasio.recoverPos, rasio.fatalPos,], color=['red','green'])

line_rasio.line(rasio.Tanggal,rasio.recoverPos,legend_label="Revovery / Positif",line_width=1,line_color='green')
line_rasio.line(rasio.Tanggal,rasio.fatalPos,legend_label="Fatal / Positif",line_width=1,line_color='darkorange')
line_rasio.line(rasio.Tanggal,rasio.rec_fat,legend_label="Recovery / Fatal",line_width=1,line_color='red')
line_rasio.legend.label_text_font_size = '9pt'

line_rasio.add_tools(HoverTool(
        tooltips=[('Tanggal', "@x{%m-%d-%Y}"),('Jumlah', "@y{0}")],
        formatters={
            'x': 'datetime',
            'Value': 'printf',
        }))
show(line_rasio)


# PENGOLAHAN KABUPATEN & KOTA SE PROVINSI

In [87]:
daerah2=df.loc[df['Tanggal']=='4/18/2020'].sort_values(by=['PDP'],ascending=True)['Daerah']
pdp2=df.loc[df['Tanggal']=='4/18/2020'].sort_values(by=['PDP'],ascending=True)['PDP']
daerah3=df.loc[df['Tanggal']=='4/25/2020'].sort_values(by=['PDP'],ascending=True)['Daerah']
pdp3=df.loc[df['Tanggal']=='4/25/2020'].sort_values(by=['PDP'],ascending=True)['PDP']

In [88]:
bar = figure(
  y_range=daerah3,
  title = 'Urutan Kabupaten/Kota Berdasarkan Jumlah Kasus PDP per '+tgl+' 2020',
  x_axis_label ='Kab/Kota',
  plot_width=800,
  plot_height=600,tools=''
)
bar.hbar(
    y=daerah3,
    right=pdp3,
    left=0,
    height=0.4,
    color='orange',
    fill_alpha=0.5
)
show(bar)

bar2 = figure(
  y_range=daerah2,
  title = 'Urutan Kabupaten/Kota Berdasarkan Jumlah Kasus PDP per '+tgl+' 2020',
  x_axis_label ='Kab/Kota',
  plot_width=800,
  plot_height=600,tools=''
)
bar2.hbar(
    y=daerah2,
    right=pdp2,
    left=0,
    height=0.4,
    color='orange',
    fill_alpha=0.5
)
show(bar2)


In [89]:
daerah2=df.loc[df['Tanggal']=='4/18/2020'].sort_values(by=['ODP'],ascending=True)['Daerah']
odp2=df.loc[df['Tanggal']=='4/18/2020'].sort_values(by=['ODP'],ascending=True)['ODP']

daerah3=df.loc[df['Tanggal']=='4/25/2020'].sort_values(by=['ODP'],ascending=True)['Daerah']
odp3=df.loc[df['Tanggal']=='4/25/2020'].sort_values(by=['ODP'],ascending=True)['ODP']

bar3 = figure(
  y_range=daerah3,
  title = 'Urutan Kabupaten/Kota Berdasarkan Jumlah Kasus ODP per '+tgl+' 2020',
  x_axis_label ='Kab/Kota',
  plot_width=800,
  plot_height=600,tools=''
)
bar3.hbar(
    y=daerah3,
    right=odp3,
    left=0,
    height=0.4,
    color='orange',
    fill_alpha=0.5
)
show(bar3)

bar4 = figure(
  y_range=daerah2,
  title = 'Urutan Kabupaten/Kota Berdasarkan Jumlah Kasus ODP per '+tgl+' 2020',
  x_axis_label ='Kab/Kota',
  plot_width=800,
  plot_height=600,tools=''
)
bar4.hbar(
    y=daerah2,
    right=odp2,
    left=0,
    height=0.4,
    color='orange',
    fill_alpha=0.5
)
show(bar4)


In [90]:
import folium
import pandas as pd
import geopandas as gpd
from shapely.geometry import Polygon
geojson="../data/kabkota.geojson"
gdf = gpd.read_file(geojson)
gdf.head()
peta_kabkota = folium.Map(location=[2.3330929,99.2180891], zoom_start=8)

folium.GeoJson(gdf, 
        highlight_function=lambda x: {'weight':0.4,'color':'white',
              'fillOpacity':0.4},
        style_function=lambda x: {'fillOpacity' : 0.7, 'color': 'white',
                                   'fillColor': 'red' if
                                   x['properties']['BAHAYA']=='Tinggi' 
                                  else 'orange' if x['properties']['BAHAYA']=='Agak Tinggi'
                                  else 'yellow' if x['properties']['BAHAYA']=='Sedang'
                                  else 'green'},
        tooltip=folium.features.GeoJsonTooltip(
                fields=['DAERAH','ODP','PDP','POSITIF','SEMBUH','MENINGGAL'],
                aliases=['Kab/Kota: ','ODP: ','PDP: ','Positif: ','Sembuh: ','Meninggal: '],
                style=("background-color: white; color: #333333; font-family: arial; font-size: 12px; line-height: 80%;") 
        )).add_to(peta_kabkota)

peta_kabkota

In [59]:
gdf

,KODE,ODP,PDP,POSITIF,SEMBUH,MENINGGAL,BAHAYA,DAERAH,geometry
0,0201,27,81,70,8,7,Tinggi,Kota Medan,"MULTIPOLYGON (((98.70711 3.77194, 98.70512 3.7..."
1,0202,3,4,7,1,0,Tinggi,Kota Pematangsiantar,"MULTIPOLYGON (((99.09980 3.01859, 99.09331 3.0..."
2,0203,3,1,0,0,0,Agak Tinggi,Kota Tanjung Balai,"MULTIPOLYGON (((99.81766 2.92479, 99.81532 2.9..."
3,0204,279,0,1,1,0,Tinggi,Kota Binjai,"MULTIPOLYGON (((98.49850 3.56468, 98.49754 3.5..."
4,0205,601,2,0,0,0,Agak Tinggi,Kota Tebingtinggi,"MULTIPOLYGON (((99.17355 3.37373, 99.17684 3.3..."
5,0206,11,0,0,0,0,Sedang,Kota Sibolga,"MULTIPOLYGON (((98.12274 1.45272, 98.11611 1.4..."
6,0207,6,1,0,0,0,Agak Tinggi,Kota Padangsidimpuan,"MULTIPOLYGON (((99.34171 1.41224, 99.35212 1.3..."
7,0208,41,18,8,2,2,Tinggi,Kab. Deli Serdang,"MULTIPOLYGON (((98.71008 3.79449, 98.70714 3.7..."
8,0209,10,1,0,0,0,Agak Tinggi,Kab. Langkat,"MULTIPOLYGON (((98.20419 4.11957, 98.19511 4.1..."
9,0211,33,19,12,0,0,Tinggi,Kab. Simalungun,"MULTIPOLYGON (((99.56136 3.07101, 99.54835 3.0..."
